#### 0. import required functionality


In [36]:
import pandas as pd
import plotly.express as px

pd.options.plotting.backend = 'plotly'

1. read data from csv


In [37]:
sales = pd.read_csv('CustomerSales.csv', usecols=[0,4,5,10,13,15,16,17,18], parse_dates=[3])
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 530 entries, 0 to 529
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   bestelnr         530 non-null    int64         
 1   woonplaats       530 non-null    object        
 2   regioCode        530 non-null    object        
 3   besteldatum      530 non-null    datetime64[ns]
 4   aantal           530 non-null    int64         
 5   omschrijving     530 non-null    object        
 6   catomschrijving  530 non-null    object        
 7   prijs            530 non-null    float64       
 8   regelOmzet       530 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 37.4+ KB


Remarks:
- (1) Keep in mind that parameters are evaluated left to right in their definition order. Here it means that in `parse_dates=[3]`, 3 refers to the third column after `usecols` has been applied. Even if you put `parse_dates` before `usecols` in the call (which is perfectly fine when using named parameters), you have to refer to the column number after the `uscols` has been applied, because `parse_dates` is defined after `usecols` in the function definition of `read_csv`.

In [38]:
sales.columns

Index(['bestelnr', 'woonplaats', 'regioCode', 'besteldatum', 'aantal',
       'omschrijving', 'catomschrijving', 'prijs', 'regelOmzet'],
      dtype='object')

#### 2. inspect data


#### 3. install SQL magic


In [39]:
%load_ext sql

%config SqlMagic

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
SqlMagic(Magics, Configurable) options
------------------------------------
SqlMagic.autocommit=<Bool>
    Set autocommit mode
    Current: True
SqlMagic.autolimit=<Int>
    Automatically limit the size of the returned result sets
    Current: 0
SqlMagic.autopandas=<Bool>
    Return Pandas DataFrames instead of regular result sets
    Current: True
SqlMagic.column_local_vars=<Bool>
    Return data into local variables from column names
    Current: False
SqlMagic.displaycon=<Bool>
    Show connection string after execute
    Current: False
SqlMagic.displaylimit=<Int>
    Automatically limit the number of rows displayed (full result set is still
    stored)
    Current: None
SqlMagic.dsn_filename=<Unicode>
    Path to DSN file. When the first argument is of the form [section], a
    sqlalchemy connection string is formed from the matching section in the DSN
    file.
    Current: 'odbc.ini'
SqlMagic.feedback=<Bool

In [40]:
%config SqlMagic.autopandas=True
%config SqlMagic.displaycon=False
%config SqlMagic.feedback=False

#### 4. configure DuckDB


In [41]:
%sql duckdb://

Remarks: you need to have installed duckdb-engine (in addition to duckdb) for this to work. The SqlMagic extension accepts SqlAlchemy URI's and duckdb-engine installs a SqlAlchemy engine.

In [42]:
%sql select * from sales using sample 5

,bestelnr,woonplaats,regioCode,besteldatum,aantal,omschrijving,catomschrijving,prijs,regelOmzet
0,1318,Veghel,GVGH,2014-01-25,13,Soja Chocolademelk Biologisch,biologische artikelen,17.50,227.5
1,1129,Waalre,REHV,2012-10-31,10,Magere Kwark,zuivelartikelen,8.95,89.5
2,1107,Laarbeek,RHMD,2012-09-21,11,Ganzenlever,luxe artikelen van de traitteur,83.60,919.6
3,1351,Erp,GVGH,2014-05-18,24,Vruchtenyoghurt Mango,zuivelartikelen,16.25,390.0
4,1410,Veghel,GVGH,2014-09-13,6,Vruchtenyoghurt Aardbei,zuivelartikelen,15.25,91.5


#### 5. how many lines have our orders?


Our approach is that we solve a problem in SQL first and then try to translate that solution to pandas. That does not always lead to the shortest or (if it exists) most idiomatic solution, but it enables us to leverage on our SQL knowledge which can help you substantially in learning pandas.

In [43]:
%%sql
select nrlines, count(*) as aantal
from (
    select bestelnr, count(*) as nrlines
    from sales
    group by bestelnr
)
group by nrlines

,nrlines,aantal
0,1,387
1,2,27
2,3,14
3,4,9
4,5,1
5,6,1


In [44]:
(
    sales
    .groupby('bestelnr', as_index=False)
    .size()
    .groupby('size')
    .size()
)

size
1    387
2     27
3     14
4      9
5      1
6      1
dtype: int64

We use the method chaining style in our solutions because it has several benefits:
- it enables and clearly visualizes a step-wise approach (transformation pipeline);
- it prevents us to make overly use of named intermediate results;
- it easier on the eyes.

Perhaps a more idiomatic way to solve this, is making use of `value_counts`, which is a combination of `groupby` and `size`.

In [45]:
(
    sales
    .value_counts('bestelnr')
    .value_counts()
)

1    387
2     27
3     14
4      9
6      1
5      1
dtype: int64

#### 6. how many orders have been placed in each month in each year?


#### 7. what is the growth in revenue for each month, compared to the same month a year before?


#### 8. what is the growth in average daily sales for each weekday, compared to the same weekday a year before?
   (Keep in mind that days without sales contribute to the average as days with zero sales.)


#### 9. what is for each year, for each weekday the number of weeks in which it was the day with the highest sales?


In [46]:
%%sql df <<
with cte as (
    select besteldatum, rank() over w as rnk
    from sales
    group by besteldatum
    window w as (
        partition by year(besteldatum), week(besteldatum)
        order by sum(regelOmzet) desc
    )
)
select year(besteldatum) as yr, strftime(besteldatum, '%A') as day, count(*) as nr
from cte
where rnk = 1
group by year(besteldatum), strftime(besteldatum, '%A'), isodow(besteldatum)
order by yr, isodow(besteldatum)


Returning data to local variable df


Remarks: In the first step (cte) we summarize sales on a daily grain and than determine for each day its rank in the revenue order of the week in the year the day belongs to. In the second step we filter the week winners and count the winning day names for each week.

Because we want to visualize our results and do some presentation related post processing, we have captured our result set in a pandas dataframe. 

In [47]:
df

,yr,day,nr
0,2012,Monday,10
1,2012,Tuesday,4
2,2012,Wednesday,6
3,2012,Thursday,5
4,2012,Friday,9
5,2012,Saturday,11
6,2012,Sunday,4
7,2013,Monday,5
8,2013,Tuesday,10
9,2013,Wednesday,5


In [48]:
wdays = pd.date_range(start='2022-01-03', periods=7, freq='D').day_name()

(
    df
    .set_index(['day', 'yr'])
    .unstack()
    .reindex(wdays, axis='index')
)

nr          
yr        2012 2013 2014
Monday      10    5    1
Tuesday      4   10    5
Wednesday    6    5    5
Thursday     5    7   10
Friday       9    6   11
Saturday    11    8    9
Sunday       4    7   10

Remarks:
- (1) We create an array (actually, a pandas index) that stores the day of the week in their proper order (Monday-Sunday).
- (5) In order to use `unstack` in (6) we have to set the index to day and year. (`unstack` operates on a (multi-)index.)
- (7) We re-index to sort the result in the proper order.

In [49]:
(
    df
    .astype({'yr': str})
    .plot(x='day', y='nr', color='yr', barmode='group', kind='bar')
)

In [50]:
(
    sales
    .groupby('besteldatum', as_index=False)
    .regelOmzet
    .sum()
    .assign(
        yr = lambda x: x.besteldatum.dt.year,
        week = lambda x: x.besteldatum.dt.isocalendar().week,
        rnk = lambda x: x.groupby(['yr', 'week']).regelOmzet.rank(ascending=False)
    )
    .query("rnk == 1")  # using the numexpr engine, equal to .loc[lambda x: x.rnk == 1]
    .assign(wday=lambda x: x.besteldatum.dt.day_name())
    .value_counts(['yr', 'wday'], sort=False)
    .unstack(0)
    .reindex(wdays, axis='index')
)

yr,2012,2013,2014
Monday,9,5,1
Tuesday,4,10,5
Wednesday,6,5,5
Thursday,5,7,10
Friday,9,6,11
Saturday,11,8,9
Sunday,4,7,10


With some minor adaptions, this is a rather straightforward translation of the SQL solution:
- (7,8) We have made an explicit columns for yeR and week, as we cannot easily create them in the `groupby` expression in pandas. We could have combined both `assign` methods and spare some addional line, but it would have offended our _stick to the SQL solution_ approach.
- (9) This is the translation of the SQL window function. I would argue that it is easier to understand because less _ceremoniously_ than the SQL window solution.

An alternative approach which we didn't cover in the workshop is perhaps a bit shorter (and more 'pandatic') but it is obviously not inspired by the SQL solution.

In [51]:
(
    sales
    .groupby('besteldatum')
    .regelOmzet
    .sum()
    .groupby([lambda x: x.year, lambda x: x.isocalendar().week])
    .idxmax()
    .dt.day_name()
    .groupby(level=0)
    .value_counts()
    .unstack(0)
    .reindex(wdays)
    .rename_axis(index="weekday", columns='year')
)

year,2012,2013,2014
weekday,,,
Monday,9,5,1
Tuesday,4,10,5
Wednesday,6,5,5
Thursday,5,7,10
Friday,9,6,11
Saturday,11,8,9
Sunday,4,7,10


Remarks:
- (5) In contrast to using lambdas in most other methods that are used in a method chain, the unbound variable of the lambda in a group by is bound to the index of the object it is called on. Here we make use of that.
- (7) `idxmax` returns the index of the line with the maximaum value. Here that index is a date, from which we retrieve the day name in (8).
- (9,10) We group on the first level of the index, which is the year and we count how many times each weekday occurs int hat year (10).